# Proceso exploratorio para la base de datos de AirBNB

## Importación de librerías

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import ast
import numpy as np
from pathlib import Path
sys.path.insert(0, str((Path.cwd() / ".." / "src").resolve()))
from extraction import Extraction

## Conexión a la base de datos

In [3]:
uri = "mongodb://localhost:27017/"
db_name = "bi_mx"

extra = Extraction() #Creación del objeto de la clase Extraction
db = extra.mongodb_connection(uri, db_name)

Conexión exitosa a la base de datos: bi_mx


## Entendimiento general de los datos

### Primeras filas de cada colección

In [4]:
#Creación de los DF de las colecciones
df_listings = extra.load_mongodb_datasets(db, 'listings_mx')
df_calendar = extra.load_mongodb_datasets(db, 'calendar_mx')
df_reviews = extra.load_mongodb_datasets(db, 'reviews_mx')

#Cerrar la conexión con la base de datos
extra.close_mongodb_connection(uri)

In [4]:
#Obtención de las primeras filas de la colección listings
df_listings.head()

,_id,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,68b1b29599c73fe63e06d79b,35797,https://www.airbnb.com/rooms/35797,20250319150644,2025-03-21,city scrape,Villa Dante,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Shopping Mall, Interlomas Park and th...",https://a0.muscache.com/pictures/f395ab78-1185...,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68b1b29599c73fe63e06d79c,44616,https://www.airbnb.com/rooms/44616,20250319150644,2025-03-20,previous scrape,Condesa Haus,A new concept of hosting in mexico through a b...,NaN,https://a0.muscache.com/pictures/251410/ec75fe...,...,2011-11-09,2025-01-01,4.59,4.56,4.70,4.87,4.78,4.98,4.47,0.40
2,68b1b29599c73fe63e06d79d,56074,https://www.airbnb.com/rooms/56074,20250319150644,2025-03-20,city scrape,Great space in historical San Rafael,This great apartment is located in one of the ...,Very traditional neighborhood with all service...,https://a0.muscache.com/pictures/3005118/60dac...,...,2011-04-02,2025-02-27,4.87,4.95,4.88,4.98,4.94,4.76,4.79,0.49
3,68b1b29599c73fe63e06d79e,67703,https://www.airbnb.com/rooms/67703,20250319150644,2025-03-20,previous scrape,"2 bedroom apt. deco bldg, Condesa","Comfortably furnished, sunny, 2 bedroom apt., ...",NaN,https://a0.muscache.com/pictures/3281720/6f078...,...,2011-11-17,2024-10-30,4.90,4.82,4.76,4.94,4.92,4.98,4.92,0.31
4,68b1b29599c73fe63e06d79f,70644,https://www.airbnb.com/rooms/70644,20250319150644,2025-03-22,city scrape,Beautiful light Studio Coyoacan- full equipped !,COYOACAN designer studio quiet & safe! well eq...,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,...,2012-02-14,2024-12-28,4.92,4.91,4.96,4.96,4.98,4.96,4.92,0.83


In [5]:
#Obtención de las primeras filas de la colección calendar
df_calendar.head()

,_id,listing_id,date,available,price,minimum_nights,maximum_nights,adjusted_price
0,68ba3ad4c2b559caebfeba1e,287940,2025-03-20,False,$50.00,1.0,1125.0,NaN
1,68ba3ad4c2b559caebfeba1f,287940,2025-03-21,False,$50.00,1.0,1125.0,NaN
2,68ba3ad4c2b559caebfeba20,287940,2025-03-22,False,$50.00,1.0,1125.0,NaN
3,68ba3ad4c2b559caebfeba21,287940,2025-03-23,False,$50.00,1.0,1125.0,NaN
4,68ba3ad4c2b559caebfeba22,287940,2025-03-24,False,$50.00,1.0,1125.0,NaN


In [6]:
#Obtención de las primeras filas de la colección reviews
df_reviews.head()

,_id,listing_id,id,date,reviewer_id,reviewer_name,comments
0,68ba3a56c2b559caebeaa58b,44616,706908,2011-11-09,634733,Lindsay,Forget staying in a hotel. Stay at condesa hau...
1,68ba3a56c2b559caebeaa58c,44616,2006160,2012-08-16,3087087,Samuel,"Fantastic location, great place, friendly host..."
2,68ba3a56c2b559caebeaa58d,44616,3174954,2012-12-28,3234920,Anna,I would highly recommend Condesa Haus for anyo...
3,68ba3a56c2b559caebeaa58e,44616,3271579,2013-01-04,2199822,Shelley And Pall,Great stay. Thanks. Highly recommend.
4,68ba3a56c2b559caebeaa58f,44616,3841065,2013-03-19,2423825,Leonardo,This was not a very good experience I am afrai...


### Cantidad de registros y columnas

In [7]:
#Colección listings
n_rows, n_columns = df_listings.shape
print(f"Número de registros de la colección listings: {n_rows}")
print(f"Número de columnas de la colección listings: {n_columns}\n")

#Colección calendar
n_rows, n_columns = df_calendar.shape
print(f"Número de registros de la colección calendar: {n_rows}")
print(f"Número de columnas de la colección calendar: {n_columns}\n")

#Colección reviews
n_rows, n_columns = df_reviews.shape
print(f"Número de registros de la colección reviews: {n_rows}")
print(f"Número de columnas de la colección reviews: {n_columns}")

Número de registros de la colección listings: 26067
Número de columnas de la colección listings: 77

Número de registros de la colección calendar: 9514717
Número de columnas de la colección calendar: 8

Número de registros de la colección reviews: 1315986
Número de columnas de la colección reviews: 7


### Verificar tipos de datos

In [8]:
#Colección listings
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26067 entries, 0 to 26066
Data columns (total 77 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   _id                                           26067 non-null  object        
 1   id                                            26067 non-null  int64         
 2   listing_url                                   26067 non-null  object        
 3   scrape_id                                     26067 non-null  int64         
 4   last_scraped                                  26067 non-null  datetime64[ns]
 5   source                                        26067 non-null  object        
 6   name                                          26067 non-null  object        
 7   description                                   25300 non-null  object        
 8   neighborhood_overview                         14391 non-null  obje

In [9]:
#Colección calendar
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9514717 entries, 0 to 9514716
Data columns (total 8 columns):
 #   Column          Dtype         
---  ------          -----         
 0   _id             object        
 1   listing_id      int64         
 2   date            datetime64[ns]
 3   available       bool          
 4   price           object        
 5   minimum_nights  float64       
 6   maximum_nights  float64       
 7   adjusted_price  object        
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 517.2+ MB


In [10]:
#Colección reviews
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315986 entries, 0 to 1315985
Data columns (total 7 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   _id            1315986 non-null  object        
 1   listing_id     1315986 non-null  int64         
 2   id             1315986 non-null  int64         
 3   date           1315986 non-null  datetime64[ns]
 4   reviewer_id    1315986 non-null  int64         
 5   reviewer_name  1315980 non-null  object        
 6   comments       1315986 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 70.3+ MB


## Calidad de los datos

### Valores nulos

#### Colección listings

In [11]:
#Sumatoria de nulos
df_listings.isnull().sum()

_id                               0
id                                0
listing_url                       0
scrape_id                         0
last_scraped                      0
                               ... 
review_scores_checkin          3262
review_scores_communication    3262
review_scores_location         3262
review_scores_value            3262
reviews_per_month              3261
Length: 77, dtype: int64

In [17]:
#Revisar columnas específicas con valores nulos
cols_con_nulos = df_listings.columns[df_listings.isnull().any()].tolist()
print(cols_con_nulos)

['description', 'neighborhood_overview', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_listings_count', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'price', 'has_availability', 'estimated_revenue_l365d', 'host_neighbourhood', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']


In [16]:
#Porcentaje de nulos
df_listings.isnull().mean()*100

_id                             0.000000
id                              0.000000
listing_url                     0.000000
scrape_id                       0.000000
last_scraped                    0.000000
                                 ...    
review_scores_checkin          12.513906
review_scores_communication    12.513906
review_scores_location         12.513906
review_scores_value            12.513906
reviews_per_month              12.510070
Length: 77, dtype: float64

In [ ]:
#Gráfica de nulos por columna
sns.heatmap(df_listings.isna(), cbar = False, cmap = 'viridis')
plt.title('Valores nulos')
plt.show()

#### Colección calendar

In [5]:
#Sumatoria de nulos
df_calendar.isnull().sum()

_id                     0
listing_id              0
date                    0
available               0
price                   0
minimum_nights        220
maximum_nights        220
adjusted_price    9512527
dtype: int64

In [6]:
#Porcentaje de nulos
df_calendar.isnull().mean()*100

_id                0.000000
listing_id         0.000000
date               0.000000
available          0.000000
price              0.000000
minimum_nights     0.002312
maximum_nights     0.002312
adjusted_price    99.976983
dtype: float64

In [ ]:
#Gráfica de nulos por columna
sns.heatmap(df_calendar.isna(), cbar = False, cmap = 'viridis')
plt.title('Valores nulos')
plt.show()

#### Colección reviews

In [ ]:
#Sumatoria de nulos
df_reviews.isnull().sum()

In [ ]:
#Porcentaje de nulos
df_reviews.isnull().mean()*100

In [ ]:
#Gráfica de nulos por columna
sns.heatmap(df_reviews.isna(), cbar = False, cmap = 'viridis')
plt.title('Valores nulos')
plt.show()

### Valores duplicados

In [ ]:
#Colección listings
bad_cols = [c for c in df_listings.columns
            if df_listings[c].apply(lambda x: isinstance(x, (list, dict, set))).any()]

good_cols = [c for c in df_listings.columns if c not in bad_cols]

dupli_rows = df_listings[df_listings.duplicated(subset=good_cols)]
print(f"Registros duplicados de la colección listings: {dupli_rows.shape[0]}")

#Colección calendar
dupli_rows = df_calendar[df_calendar.duplicated()] 
print(f"Registros duplicados de la colección calendar: {dupli_rows.shape[0]}")

#Colección reviews
dupli_rows = df_reviews[df_reviews.duplicated()] 
print(f"Registros duplicados de la colección reviews: {dupli_rows.shape[0]}")

Dado que en ninguna de las colecciones parece haber datos duplicados, no hay necesidad de hacer alguna operación en este sentido.

### Posibles valores atípicos

#### Colección listings

Antes de poder buscar valores atípicos, primero se deben limpiar algunos campos numéricos. Los cuales no están en un formato correcto y tienen caracteres que no deberían.

In [ ]:
#Copiar el dataset de listings para hacer las cambios 
df_listings_copy = df_listings.copy()

In [ ]:
#Cambiar columna price a tipo float
df_listings_copy['price'] = (df_listings_copy['price'].astype(str)
                   .str.replace(r'[^0-9.\-]', '', regex=True)
                   .replace('', np.nan).astype(float))

#Cambiar columnas host_response_rate y host_acceptance_rate a tipo float
df_listings_copy["host_response_rate"] = (
    df_listings_copy["host_response_rate"]
    .replace("N/A", None)              # Cambiar "N/A" a None
    .str.replace("%", "", regex=False) # Quitar el símbolo %
    .astype(float)                     # Convertir a float
)

df_listings_copy["host_acceptance_rate"] = (
    df_listings_copy["host_acceptance_rate"]
    .replace("N/A", None)
    .str.replace("%", "", regex=False)
    .astype(float)
)

In [ ]:
#Boxplot precio
df_listings_copy.boxplot(column='price')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_listings_copy.loc[df_listings_copy['price'].idxmax(),['name', 'host_name', 'price']])

Price presenta valores atípicos. Esto es debido a que, analizando el nombre del valor máximo, por ejemplo, es una propiedad que solo está disponible para los tiempos navideños. Lo cual, a menos de que se quisiera saber información relacionada con esto, lo único que hace es generar ruido en los datos. Sin embargo, sería apropiado evaluar si es necesario eliminarlo o no.

In [ ]:
#Boxplot accommodates
df_listings_copy.boxplot(column='accommodates')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_listings_copy.loc[df_listings_copy['accommodates'].idxmax(),['name', 'host_name', 'accommodates']])

Accommodates presenta valores atípicos, pero sería apropiado analizar si estos deberían eliminarse.

In [ ]:
#Boxplot bedrooms
df_listings_copy.boxplot(column='bedrooms')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_listings_copy.loc[df_listings_copy['bedrooms'].idxmax(),['name', 'host_name', 'bedrooms']])

Bedrooms presenta valores atípicos, aunque puede no ser necesario eliminarlos.

In [ ]:
#Boxplot beds
df_listings_copy.boxplot(column='beds')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_listings_copy.loc[df_listings_copy['beds'].idxmax(),['name', 'host_name', 'beds']])

Beds presenta valores atípicos, aunque puede no ser necesario eliminarlos.

In [ ]:
#Boxplot bathrooms
df_listings_copy.boxplot(column='bathrooms')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_listings_copy.loc[df_listings_copy['bathrooms'].idxmax(),['name', 'host_name', 'bathrooms']])

Bathrooms presenta valores atípicos, pero puede no ser necesario eliminarlos.

In [ ]:
#Boxplot minimum_nights
df_listings_copy.boxplot(column='minimum_nights')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_listings_copy.loc[df_listings_copy['minimum_nights'].idxmax(),['name', 'host_name', 'minimum_nights']])

Minimum_nights presenta valores atípicos, pero puede no ser necesario eliminarlos.

In [ ]:
#Boxplot maximum_nights
df_listings_copy.boxplot(column='maximum_nights')
plt.show()

Maximum_nights no presenta valores atípicos

In [ ]:
#Boxplot availability_365
df_listings_copy.boxplot(column='availability_365')
plt.show()

Availability_365 no presenta valores atípicos

In [ ]:
#Boxplot number_of_reviews
df_listings_copy.boxplot(column='number_of_reviews')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_listings_copy.loc[df_listings_copy['number_of_reviews'].idxmax(),['name', 'host_name', 'number_of_reviews']])

Number_of_reviews presenta valores atípicos, pero no es necesario eliminarlos.

In [ ]:
#Boxplot review_scores_rating
df_listings_copy.boxplot(column='review_scores_rating')
plt.show()

In [ ]:
#Revisar el valor mínimo de la columna
print("Mínimo:", df_listings_copy.loc[df_listings_copy['review_scores_rating'].idxmin(),['name', 'host_name', 'review_scores_rating']])

Review_scores_rating tiene valores atípicos que, en este caso, son negativos. Sin embargo, puede no ser necesario eliminarlos.

In [ ]:
#Boxplot review_scores_accuracy
df_listings_copy.boxplot(column='review_scores_accuracy')
plt.show()

In [ ]:
#Revisar el valor mínimo de la columna
print("Mínimo:", df_listings_copy.loc[df_listings_copy['review_scores_accuracy'].idxmin(),['name', 'host_name', 'review_scores_accuracy']])

Review_scores_accuracy tiene valores atípicos que, en este caso, son negativos. Sin embargo, puede no ser necesario eliminarlos.

In [ ]:
#Boxplot review_scores_cleanliness
df_listings_copy.boxplot(column='review_scores_cleanliness')
plt.show()

In [ ]:
#Revisar el valor mínimo de la columna
print("Mínimo:", df_listings_copy.loc[df_listings_copy['review_scores_cleanliness'].idxmin(),['name', 'host_name', 'review_scores_cleanliness']])

Review_scores_cleanliness tiene valores atípicos que, en este caso, son negativos. Sin embargo, puede ser necesario eliminarlos.

In [ ]:
#Boxplot review_scores_communication
df_listings_copy.boxplot(column='review_scores_communication')
plt.show()

In [ ]:
#Revisar el valor mínimo de la columna
print("Mínimo:", df_listings_copy.loc[df_listings_copy['review_scores_communication'].idxmin(),['name', 'host_name', 'review_scores_communication']])

Review_scores_communication tiene valores atípicos que, en este caso, son negativos. Sin embargo, puede no ser necesario eliminarlos.

In [ ]:
#Boxplot review_scores_location
df_listings_copy.boxplot(column='review_scores_location')
plt.show()

In [ ]:
#Revisar el valor mínimo de la columna
print("Mínimo:", df_listings_copy.loc[df_listings_copy['review_scores_location'].idxmin(),['name', 'host_name', 'review_scores_location']])

Review_scores_location tiene valores atípicos que, en este caso, son negativos. Sin embargo, puede no ser necesario eliminarlos.

In [ ]:
#Boxplot review_scores_value
df_listings_copy.boxplot(column='review_scores_value')
plt.show()

In [ ]:
#Revisar el valor mínimo de la columna
print("Mínimo:", df_listings_copy.loc[df_listings_copy['review_scores_value'].idxmin(),['name', 'host_name', 'review_scores_value']])

Review_scores_value tiene valores atípicos que, en este caso, son negativos. Sin embargo, puede no ser necesario eliminarlos.

In [ ]:
#Boxplot host_response_rate
df_listings_copy.boxplot(column='host_response_rate')
plt.show()

In [ ]:
#Revisar el valor mínimo de la columna
print("Mínimo:", df_listings_copy.loc[df_listings_copy['host_response_rate'].idxmin(),['name', 'host_name', 'host_response_rate']])

Host_response_rate tiene valores atípicos que, en este caso, son negativos. Sin embargo, puede no ser necesario borrarlos.

In [ ]:
#Boxplot host_acceptance_rate
df_listings_copy.boxplot(column='host_acceptance_rate')
plt.show()

In [ ]:
#Revisar el valor mínimo de la columna
print("Mínimo:", df_listings_copy.loc[df_listings_copy['host_acceptance_rate'].idxmin(),['name', 'host_name', 'host_acceptance_rate']])

Host_acceptance_rate tiene valores atípicos que, en este caso, son negativos. Sin embargo, puede no ser necesario eliminarlos.

#### Colección calendar

Antes de poder buscar valores atípicos, primero se deben limpiar algunos campos numéricos. Los cuales no están en un formato correcto y tienen caracteres que no deberían.

In [ ]:
#Copiar el dataset de calendar para hacer las cambios 
df_calendar_copy = df_calendar.copy()

In [ ]:
#Cambiar columna price a tipo float
df_calendar_copy['price'] = (df_calendar_copy['price'].astype(str)
                   .str.replace(r'[^0-9.\-]', '', regex=True)
                   .replace('', np.nan).astype(float))

In [ ]:
#Boxplot precio
df_calendar_copy.boxplot(column='price')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_calendar_copy.loc[df_calendar_copy['price'].idxmax(),['listing_id', 'price']])

Price presenta valores atípicos, pero siendo que es el precio de una reserva existente (son muchos registros con estos mismos valores, pero diferentes fechas), no sería conveniente borrarlo. Aunque vale la pena destacar el hecho de que ese id no aparece en la colección de listings, por lo que se debería evaluar de cerca este escenario y revisar la relevancia de estos documentos.

In [ ]:
#Boxplot minimum_nights
df_calendar_copy.boxplot(column='minimum_nights')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_calendar_copy.loc[df_calendar_copy['minimum_nights'].idxmax(),['listing_id', 'minimum_nights']])

Minimum_nights tiene valores atípicos, pero no sería necesario eliminarlos.

In [ ]:
#Boxplot maximum_nights
df_calendar_copy.boxplot(column='maximum_nights')
plt.show()

In [ ]:
#Revisar el valor máximo de la columna
print("Máximo:", df_calendar_copy.loc[df_calendar_copy['maximum_nights'].idxmax(),['listing_id', 'maximum_nights']])

Maxmimum_nights tiene valores atípicos, pero esta vez si hay que eliminarlos. Puesto que este valor no es el real que corresponde al listings_id mostrado en el registro, dado que esa propiedad tiene un máximo de nocches de 1125.

#### Colección Reviews

Esta colección no tiene campos numéricos que se puedan evaluar en busca de datos atípicos, por lo que aquí no va haber ningún análisis.

## Transformaciones potenciales

### Desanidar campos

Dentro de todas las colecciones, solo listings tiene campos que deberían ser desanidados (listas o diccionarios). Siendo en este caso, las columnas de ammenities y la de host_verification.

Primero, se procede a desanidar el campo de host_verifications. Ante de hacerlo, se debe validar qué es lo que contiene la columna.

In [ ]:
#Contar valores únicos en la columna para buscar anomalias
df_listings["host_verifications"].value_counts().head(30)

Ahora, se procede a hacer el desanidado de la columna.

In [ ]:
#Convertir la columna de string a lista
def safe_eval(x): # Función para hacer la conversión
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return []   # si hay error, devuelve lista vacía

df_listings_copy["host_verifications"] = df_listings["host_verifications"].apply(safe_eval)

#Convertir las listas de host_verifications en varias columnas independientes
df_expanded_hv = df_listings_copy["host_verifications"].apply(pd.Series)

#Renombrar las columnas creadas
df_expanded_hv.columns = [f"host_verification_{i+1}" for i in range(df_expanded_hv.shape[1])]

#Agregar estas columnas al dataset de listings
df_listings_copy_hv = pd.concat([df_listings_copy, df_expanded_hv], axis=1)

#Eliminar la columna vieja de host_verifications
df_listings_copy_hv = df_listings_copy_hv.drop(columns=["host_verifications"])

#Reorganizar columnas nuevas para que esten en la posición de la versión antigua
col1 = df_listings_copy_hv.pop("host_verification_1") #Sacar la columna 1 de verificación
df_listings_copy_hv.insert(24, "host_verification_1", col1) #Insertarla en la posición 24

col2 = df_listings_copy_hv.pop("host_verification_2") #Sacar la columna 2 de verificación
df_listings_copy_hv.insert(25, "host_verification_2", col2) #Insertarla en la posición 25

col3 = df_listings_copy_hv.pop("host_verification_3") #Sacar la columna 3 de verificación
df_listings_copy_hv.insert(26, "host_verification_3", col3) #Insertarla en la posición 26

In [ ]:
#Revisión del resultado de la operación
df_listings_copy_hv.info()

Comparando el resultado con la versión original:

In [ ]:
df_listings_copy_hv[['host_verification_1','host_verification_2','host_verification_3']]

In [ ]:
df_listings_copy['host_verifications']

Se puede apreciar que el cambio no afectó los datos.

Ahora, pasando a la columna de amenities:

In [ ]:
#Confirmar que amenities es una columna tipo lista
print(type(df_listings_copy_hv["amenities"].iloc[0]))

In [ ]:
#Expandir amenities en varias columnas
df_expanded_a = df_listings_copy_hv["amenities"].apply(pd.Series)
df_expanded_a.columns = [f"amenity_{i+1}" for i in range(df_expanded_a.shape[1])]

#Agregar y organizar las nuevas columnas al dataset
pos = 38
df_listings_copy_hv_a = pd.concat([df_listings_copy_hv.iloc[:, :pos], df_expanded_a, df_listings_copy_hv.iloc[:, pos:]], axis=1)

In [ ]:
#Resultado de la operación
df_listings_copy_hv_a.info()